## Label doc by extracted words

## Load words

In [33]:

from nltk.corpus import stopwords as nltk_stop_words

def clean_word(s: str):
    return re.sub('[^a-zа-я]', '', s.lower())

def read_lines(fname: str):
    return open(fname, "r", 1000, "utf-8").readlines()

# Load data and shit
# Russian first names
# TODO : Russian city names in English
first_names = read_lines('./data/fnames.csv')
# Russian last names
last_names = read_lines('./data/lnames.csv')
# Russian middle names
mid_names = read_lines('./data/mnames.csv')
# Russian first names transcripted
first_names_lat = read_lines('./data/fnames_en.csv')
# Russian last names transcripted
last_names_lat = read_lines('./data/lnames_en.csv')
# Russian middle names transcripted
mid_names_lat = read_lines('./data/mnames_en.csv')
# Russian regions lemized
regions = read_lines('./data/ru-reg-tokens.csv')
# Russian settlements
settlements = read_lines('./data/ru-set-tokens.csv')

stop_words_raw = first_names + last_names + mid_names + \
    first_names_lat + last_names_lat + mid_names_lat + \
    regions + settlements

stop_words = [clean_word(x) for x in stop_words_raw]

nltk_stop_words_en = nltk_stop_words.words('english')
nltk_stop_words_ru = nltk_stop_words.words('russian')

all_stop_words_set = set(stop_words + nltk_stop_words_en + nltk_stop_words_ru)


## Load words

In [37]:
words = read_lines("./out/passport-1-words.txt")

## Cleanup words

In [41]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('russian')

def process_words(words: str):
    return [w for w in words if len(w) > 1 and w not in all_stop_words_set]

def prepare_words(words: list[str]) -> list[str]:
    # clean words
    words = [clean_word(w) for w in words]
    # process raw words
    words = process_words(words)
    # process stemmed once again
    words = [stemmer.stem(w) for w in words]
    words = process_words(words)
    return words

words = prepare_words(words)
words

['отдел',
 'уфмс',
 'област',
 'сл',
 'выдач',
 'подраздел',
 'анчн',
 'подп',
 'сл',
 'отчеств',
 'муж',
 'рожд',
 'mecro',
 'рожд',
 'горзлатоуст',
 'обл',
 'pnrusputilovmaksimaleksandrovi',
 'rusm']

## Predict doc type by prepared words

[load trained model](https://scikit-learn.org/stable/modules/model_persistence.html)

+ 0 - passport
+ 1 - driver license


In [39]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from joblib import load

clf = load('./data/trained.joblib') 
vocab = load('./data/vocab.joblib') 
countVectorizer = CountVectorizer(vocabulary=vocab)

def label_doc(words: list[str]) -> int:    
    sent = ' '.join(words)
    X = countVectorizer.fit_transform([sent]).toarray()
    return clf.predict(X)[0]

label_doc(words) 

0